# Facial Keypoints Detection

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Data Loading

In [3]:
from fastai.vision import *

In [4]:
path = Config.data_path()/'facial-keypoints-detection'
path.mkdir(parents=True, exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/facial-keypoints-detection')

In [2]:
#! pip install kaggle --upgrade

In [21]:
! rm {path}/*

In [22]:
! kaggle competitions download -c facial-keypoints-detection -p {path}

! unzip -q -n {path}/facial-keypoints-detection.zip -d {path}
! unzip -q -n {path}/training.zip -d {path}
! unzip -q -n {path}/test.zip -d {path}
! rm {path}/*.zip

path.ls()

 85%|████████████████████████████████▎     | 65.0M/76.3M [00:00<00:00, 33.0MB/s]
100%|███████████████████████████████████████| 76.3M/76.3M [00:00<00:00, 126MB/s]


[PosixPath('/home/jupyter/.fastai/data/facial-keypoints-detection/test.csv'),
 PosixPath('/home/jupyter/.fastai/data/facial-keypoints-detection/SampleSubmission.csv'),
 PosixPath('/home/jupyter/.fastai/data/facial-keypoints-detection/training.csv'),
 PosixPath('/home/jupyter/.fastai/data/facial-keypoints-detection/IdLookupTable.csv')]